In [1]:
from dotenv import load_dotenv
import os
from pathlib import Path
from pyspark.sql import SparkSession


CURRENT_DIRECTORY_NOTEBOOK = None

def intitate_notebook():
    load_dotenv()
    global CURRENT_DIRECTORY_NOTEBOOK
    if CURRENT_DIRECTORY_NOTEBOOK is None:
        os.chdir(os.getenv("PROJECT_BASE_PATH"))
        CURRENT_DIRECTORY_NOTEBOOK = Path(os.getcwd())
        print("Current directory for notebook: ", CURRENT_DIRECTORY_NOTEBOOK)
    else:
        print(
            "Current directory for notebook is already set: ",
            CURRENT_DIRECTORY_NOTEBOOK,
        )

intitate_notebook()

Current directory for notebook:  /Users/shirshmall/Personal_Drive/Credit_Risk_Model_Project


In [32]:
from notebooks.utils import column_stats, get_shape_df, get_columns_high_missing_values
from tqdm import tqdm

In [3]:
parquet_file_path = CURRENT_DIRECTORY_NOTEBOOK / Path(
    "raw_main_dataset/home-credit-credit-risk-model-stability/parquet_files/train/train_applprev_1_0.parquet"
)

spark = SparkSession.builder.appName("Parquet_File_Exploration_Static_0").getOrCreate()

df_applprev_1_0 = spark.read.parquet(str(parquet_file_path))
df_applprev_1_0.show()

24/07/17 10:56:45 WARN Utils: Your hostname, Shirshs-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.1.109 instead (on interface en0)
24/07/17 10:56:45 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/07/17 10:56:45 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/07/17 10:56:48 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------+--------------+------------+-----------------+------------------------+---------------------+------------+-----------------+--------------------------+--------------------+----------------------+---------------------+-------------------+-------------------------+---------------+-------------+------------+------------------+-------------+------------+--------------+-------------------------+---------------+-----------------+----------------+--------------------------+------------------------+-----------------+----------------+----------------------+--------------------+----------+--------------------+---------+----------------+---------------+-----------------+---------------------------+---------------------+-----------+----------+
|case_id|actualdpd_943P|annuity_853A|approvaldate_319D|byoccupationinc_3656910L|cancelreason_3545846M|childnum_21L|creationdate_885D|credacc_actualbalance_314A|credacc_credlmt_575A|credacc_maxhisbal_375A|credacc_minhisbal_90A|credacc_status_367L|cre

In [4]:
parquet_file_path = CURRENT_DIRECTORY_NOTEBOOK / Path(
    "raw_main_dataset/home-credit-credit-risk-model-stability/parquet_files/train/train_applprev_1_1.parquet"
)

spark = SparkSession.builder.appName("Parquet_File_Exploration_Static_0").getOrCreate()

df_applprev_1_1 = spark.read.parquet(str(parquet_file_path))
df_applprev_1_1.show()

+-------+--------------+------------+-----------------+------------------------+---------------------+------------+-----------------+--------------------------+--------------------+----------------------+---------------------+-------------------+-------------------------+---------------+-------------+------------+------------------+-------------+------------+--------------+-------------------------+---------------+-----------------+----------------+--------------------------+------------------------+-----------------+----------------+----------------------+--------------------+----------+--------------------+---------+----------------+---------------+-----------------+---------------------------+---------------------+-----------+----------+
|case_id|actualdpd_943P|annuity_853A|approvaldate_319D|byoccupationinc_3656910L|cancelreason_3545846M|childnum_21L|creationdate_885D|credacc_actualbalance_314A|credacc_credlmt_575A|credacc_maxhisbal_375A|credacc_minhisbal_90A|credacc_status_367L|cre

In [5]:
df_applprev_1_1.columns == df_applprev_1_0.columns

True

In [6]:
missing_values_dict = get_columns_high_missing_values(
    df_applprev_1_0, threshold=25, display=True
)

df_applprev_1_0 = df_applprev_1_0.drop(*list(missing_values_dict.keys()))

100%|██████████| 41/41 [00:07<00:00,  5.53it/s]


In [7]:
missing_values_dict = get_columns_high_missing_values(
    df_applprev_1_1, threshold=25, display=True
)

df_applprev_1_1 = df_applprev_1_1.drop(*list(missing_values_dict.keys()))

100%|██████████| 41/41 [00:06<00:00,  6.55it/s]


In [8]:
df_applprev_depth1 = df_applprev_1_0.union(df_applprev_1_1)

In [9]:
missing_values_dict = get_columns_high_missing_values(
    df_applprev_depth1, threshold=None, display=True
)

100%|██████████| 23/23 [00:05<00:00,  3.97it/s]


Column: actualdpd_943P
- Action: We can take Groupby with Average

Column: annuity_853A
- Action: We can take Groupby with Average

Column: cancelreason_3545846M
- Multiple Cancel Reasons are possible for any Case ID
- Action: Not Sure

In [41]:
from pyspark.sql.functions import countDistinct

temp_df = df_applprev_depth1.select("case_id", "cancelreason_3545846M")

# Get the distinct count of 'cancelreason_3545846M' per 'case_id'
distinct_cancel_reason_per_case_id_df = temp_df.groupBy("case_id").agg(countDistinct("cancelreason_3545846M").alias("distinct_count"))

# Collect the results into a dictionary
distinct_cancel_reason_per_case_id = {row["case_id"]: row["distinct_count"] for row in distinct_cancel_reason_per_case_id_df.collect()}

df_applprev_depth1.filter(df_applprev_depth1.case_id==1699434).show()

+-------+--------------+------------+---------------------+-----------------+--------------------+---------------+-------------+-------------+------------+---------------+--------------------------+------------------------+-----------------+----------------------+----------+---------+----------------+---------------+-----------------+---------------------------+-----------+----------+
|case_id|actualdpd_943P|annuity_853A|cancelreason_3545846M|creationdate_885D|credacc_credlmt_575A|credamount_590A|credtype_587L|district_544M|downpmt_134A|education_1138M|firstnonzeroinstldate_307D|inittransactioncode_279L|isbidproduct_390L|mainoccupationinc_437A|num_group1|pmtnum_8L|postype_4733339M|profession_152M|rejectreason_755M|rejectreasonclient_4145042M|status_219L|tenor_203L|
+-------+--------------+------------+---------------------+-----------------+--------------------+---------------+-------------+-------------+------------+---------------+--------------------------+------------------------+-

24/07/17 14:36:42 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 1066499 ms exceeds timeout 120000 ms
24/07/17 14:36:42 WARN SparkContext: Killing executors is not supported by current scheduler.
24/07/17 14:36:44 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:124)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$

In [14]:
for column_name in df_applprev_depth1.columns:
    column_stats(df_applprev_depth1, column_name)

Column Name: case_id
Minimum Value: 2
Maximum Value: 2703454
Missing Values Count: 0 (0.00%)
Number of Unique Values: 1221522
Total Number of Rows: 6525979
Column Name:  case_id
Description - Not Found
---------------------------------------------------------------------------------------------------- 




Column Name: actualdpd_943P
Minimum Value: 0.0
Maximum Value: 4206.0
Missing Values Count: 2500 (0.04%)
Number of Unique Values: 189
Total Number of Rows: 6525979
Column Name:  actualdpd_943P
Column Description:  Days Past Due (DPD) of previous contract (actual).
---------------------------------------------------------------------------------------------------- 




Column Name: annuity_853A
Minimum Value: 0.0
Maximum Value: 105130.2
Missing Values Count: 250736 (3.84%)
Number of Unique Values: 89297
Total Number of Rows: 6525979
Column Name:  annuity_853A
Column Description:  Monthly annuity for previous applications.
---------------------------------------------------------------------------------------------------- 




Column Name: cancelreason_3545846M
Minimum Value: P107_145_100
Maximum Value: a55475b1
Missing Values Count: 0 (0.00%)
Number of Unique Values: 76
Total Number of Rows: 6525979
Column Name:  cancelreason_3545846M
Column Description:  Application cancellation reason.
---------------------------------------------------------------------------------------------------- 




Column Name: creationdate_885D
Minimum Value: 2005-12-26
Maximum Value: 2020-10-19
Missing Values Count: 66 (0.00%)
Number of Unique Values: 5405
Total Number of Rows: 6525979
Column Name:  creationdate_885D
Column Description:  Date when previous application was created.
---------------------------------------------------------------------------------------------------- 




...

Column Name: mainoccupationinc_437A
Minimum Value: 0.0
Maximum Value: 199600.0
Missing Values Count: 101983 (1.56%)
Number of Unique Values: 28717
Total Number of Rows: 6525979
Column Name:  mainoccupationinc_437A
Column Description:  Client's main income amount in their previous application.
---------------------------------------------------------------------------------------------------- 


Column Name: num_group1
Minimum Value: 0
Maximum Value: 19
Missing Values Count: 0 (0.00%)
Number of Unique Values: 20
Total Number of Rows: 6525979
Column Name:  num_group1
Description - Not Found
---------------------------------------------------------------------------------------------------- 




Column Name: pmtnum_8L
Minimum Value: 3.0
Maximum Value: 63.0
Missing Values Count: 551820 (8.46%)
Number of Unique Values: 60
Total Number of Rows: 6525979
Column Name:  pmtnum_8L
Column Description:  Number of payments made for the previous application.
---------------------------------------------------------------------------------------------------- 




Column Name: postype_4733339M
Minimum Value: P140_48_169
Maximum Value: a55475b1
Missing Values Count: 0 (0.00%)
Number of Unique Values: 9
Total Number of Rows: 6525979
Column Name:  postype_4733339M
Column Description:  Type of point of sale.
---------------------------------------------------------------------------------------------------- 




Column Name: profession_152M
Minimum Value: P0_102_76
Maximum Value: a55475b1
Missing Values Count: 0 (0.00%)
Number of Unique Values: 11508
Total Number of Rows: 6525979
Column Name:  profession_152M
Column Description:  Profession of the client during their previous loan application.
---------------------------------------------------------------------------------------------------- 


Column Name: rejectreason_755M
Minimum Value: P121_60_164
Maximum Value: a55475b1
Missing Values Count: 0 (0.00%)
Number of Unique Values: 18
Total Number of Rows: 6525979
Column Name:  rejectreason_755M
Column Description:  Reason for previous application rejection.
---------------------------------------------------------------------------------------------------- 


Column Name: rejectreasonclient_4145042M
Minimum Value: P129_162_80
Maximum Value: a55475b1
Missing Values Count: 0 (0.00%)
Number of Unique Values: 14
Total Number of Rows: 6525979
Column Name:  rejectreasonclient_4145042M
Column Descri

Column Name: status_219L
Minimum Value: A
Maximum Value: T
Missing Values Count: 66 (0.00%)
Number of Unique Values: 11
Total Number of Rows: 6525979
Column Name:  status_219L
Column Description:  Previous application status.
---------------------------------------------------------------------------------------------------- 


Column Name: tenor_203L
Minimum Value: 3.0
Maximum Value: 63.0
Missing Values Count: 551820 (8.46%)
Number of Unique Values: 60
Total Number of Rows: 6525979
Column Name:  tenor_203L
Column Description:  Number of instalments in the previous application.
---------------------------------------------------------------------------------------------------- 




In [ ]:
parquet_file_path = CURRENT_DIRECTORY_NOTEBOOK / Path(
    "raw_main_dataset/home-credit-credit-risk-model-stability/parquet_files/train/train_applprev_2.parquet"
)

spark = SparkSession.builder.appName("Parquet_File_Exploration_Static_0").getOrCreate()

df_applprev_2 = spark.read.parquet(str(parquet_file_path))
df_applprev_2.show()